<a href="https://colab.research.google.com/github/diogo-costa-silva/sql-murder-mystery/blob/main/SQL_Observable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is from [here](https://observablehq.com/@observablehq/sql-mystery-game-sql-tutorial).

In [2]:
# getting the database from my github
!wget https://github.com/diogo-costa-silva/assets/raw/main/databases/sql-murder-mystery.db -O sql-murder-mystery.db

--2024-02-21 12:49:06--  https://github.com/diogo-costa-silva/assets/raw/main/databases/sql-murder-mystery.db
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/diogo-costa-silva/assets/main/databases/sql-murder-mystery.db [following]
--2024-02-21 12:49:07--  https://raw.githubusercontent.com/diogo-costa-silva/assets/main/databases/sql-murder-mystery.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3661824 (3.5M) [application/octet-stream]
Saving to: ‘sql-murder-mystery.db’

sql-murder-mystery. 100%[===================>]   3.49M  --.-KB/s    in 0.07s   

2024-02-21 12:49:07 (50.5 MB/s) - ‘sql-mur

In [3]:
# Install ipython-sql
!pip install ipython-sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.8 MB/s eta 0:00:00


In [4]:
# Load the SQL extension
%load_ext sql

In [5]:
# Create a SQLite database
%sql sqlite:///sql-murder-mystery.db

## The Prompt
A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. All the clues to this mystery are buried in a huge database, and you need to use SQL to navigate through this vast network of information.

You vaguely remember that the crime was a **murder** that occurred sometime on **Jan.15, 2018** and that it took place in **SQL City**. Start by retrieving the corresponding crime scene report from the police department’s database.

Let's see, I'll try selecting all crimes that are in SQL City on the date of the murder...

In [6]:
%%sql
select *
from crime_scene_report
where city = 'SQL City' AND date = 20180115

 * sqlite:///sql-murder-mystery.db
Done.


date,type,description,city
20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!",SQL City
20180115,assault,Report Not Found,SQL City
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


The two witnesses seem like a promising lead, i'll use the address information to find the first in the person table.

In [8]:
%%sql
select *
from person where address_street_name = 'Northwestern Dr'
order by address_number DESC
limit 10;

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
17729,Lasonya Wildey,439686,3824,Northwestern Dr,917817122
53890,Sophie Tiberio,957671,3755,Northwestern Dr,442830147
73368,Torie Thalmann,773862,3697,Northwestern Dr,341559436
96595,Coretta Cubie,303645,3631,Northwestern Dr,378403829
19420,Cody Schiel,890431,3524,Northwestern Dr,947110049
93509,Emmitt Aceuedo,916706,3491,Northwestern Dr,979073160
87456,Leonora Wolfsberger,215868,3483,Northwestern Dr,565203106
36378,Freddie Ellzey,267882,3449,Northwestern Dr,474117596
53076,Boris Bijou,664914,3327,Northwestern Dr,401191868


Then I'll look for the second by name and address, duplicating the previous query and adding the name search.

In [9]:
%%sql
select *
from person
where name LIKE '%Annabel%' AND address_street_name = 'Franklin Ave'
order by address_number DESC;

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143


Let's use their person id to look for these two people in the interviews table.

In [10]:
%%sql
select *
from interview
WHERE person_id IN (16371,14887);

 * sqlite:///sql-murder-mystery.db
Done.


person_id,transcript
14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


Ok, now let's use these very observant witnesses to search the gym membership table.

In [11]:
%%sql
select *
from get_fit_now_member
where membership_status = 'gold' AND id LIKE '48Z%'

 * sqlite:///sql-murder-mystery.db
Done.


id,person_id,name,membership_start_date,membership_status
48Z7A,28819,Joe Germuska,20160305,gold
48Z55,67318,Jeremy Bowers,20160101,gold


Then we can get the details on those two from the person table.

In [12]:
%%sql
select *
from person
WHERE id IN (28819, 67318)

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
28819,Joe Germuska,173289,111,Fisk Rd,138909730
67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


Now let's find out if one of these two checked in on January 9th as per our first witness statement.

In [14]:
%%sql
select *
from get_fit_now_check_in
WHERE membership_id in ('48Z7A', '48Z55')

 * sqlite:///sql-murder-mystery.db
Done.


membership_id,check_in_date,check_in_time,check_out_time
48Z7A,20180109,1600,1730
48Z55,20180109,1530,1700


Since they were both there, it doesn't quite narrow it down. Let's use the license plate info from the second witness to figure out if one of these two was spotted.

In [15]:
%%sql
select person.name, drivers_license.*
  from drivers_license
  join person on drivers_license.id = person.license_id
  WHERE plate_number like '%H42W%'

 * sqlite:///sql-murder-mystery.db
Done.


name,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
Tushar Chandra,664760,21,71,black,black,male,4H42WR,Nissan,Altima
Jeremy Bowers,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
Maxine Whitely,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius


Aha! I recognize that name, Jeremey Bowers is the killer!

In [17]:
%%sql
select *
from interview
WHERE person_id IN (67318)

 * sqlite:///sql-murder-mystery.db
Done.


person_id,transcript
67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


Let's take a look at the drivers license details first:

In [19]:
%%sql
select *
from drivers_license
join person on drivers_license.id = person.license_id
where car_make = 'Tesla' and car_model = 'Model S' and gender = 'female' and hair_color = 'red'

 * sqlite:///sql-murder-mystery.db
Done.


id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id_1,name,license_id,address_number,address_street_name,ssn
918773,48,65,black,red,female,917UU3,Tesla,Model S,78881,Red Korb,918773,107,Camerata Dr,961388910
291182,65,66,blue,red,female,08CM64,Tesla,Model S,90700,Regina George,291182,332,Maple Ave,337169072
202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


If we grab these 3 very similar people and see who went to the concert that might help.

In [20]:
%%sql
select *
from facebook_event_checkin
where person_id in (78881,90700,99716)

 * sqlite:///sql-murder-mystery.db
Done.


person_id,event_id,event_name,date
99716,1143,SQL Symphony Concert,20171206
99716,1143,SQL Symphony Concert,20171212
99716,1143,SQL Symphony Concert,20171229


Alright! there is only one of the 3 that went to the concert 3 times in december... let's use a join to get the name:

In [21]:
%%sql
select *
from facebook_event_checkin
join person on facebook_event_checkin.person_id = person.id
where person_id in (78881,90700,99716)

 * sqlite:///sql-murder-mystery.db
Done.


person_id,event_id,event_name,date,id,name,license_id,address_number,address_street_name,ssn
99716,1143,SQL Symphony Concert,20171206,99716,Miranda Priestly,202298,1883,Golden Ave,987756388
99716,1143,SQL Symphony Concert,20171212,99716,Miranda Priestly,202298,1883,Golden Ave,987756388
99716,1143,SQL Symphony Concert,20171229,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


And there we have it, Miranda Priestly is the mastermind!